##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN을 통한 문자열 분류

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_classification_rnn"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub)소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_classification_rnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운로드</a>
  </td>
</table>

이 텍스트 분류 튜토리얼은 감정 분석을 위한 [IMDB 대형 영화 리뷰 데이터셋](http://ai.stanford.edu/~amaas/data/sentiment/)에서 [순환 인공신경망](https://developers.google.com/machine-learning/glossary/#recurrent_neural_network)을 학습시킵니다.

## 설치

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  #%tensorflow_version은 오직 코랩(Colab)에서 존재합니다.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow_datasets as tfds
import tensorflow as tf

Import `matplotlib` and create a helper function to plot graphs:

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("에포크")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

## 입력파이프라인 설치


IMDB 대형 영화 리뷰 데이터셋은 모든 리뷰가 *긍정* 혹은 *부정*으로 이루어진 *이진 분류* 데이터셋입니다.

[TFDS](https://www.tensorflow.org/datasets)를 사용하여 데이터셋 다운로드.


In [0]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

 데이터셋 `info`는 (a tfds.features.text.SubwordTextEncoder)엔코더를 포함합니다.

In [0]:
encoder = info.features['text'].encoder

In [0]:
print ('단어 크기: {}'.format(encoder.vocab_size))

이 텍스트 인코더는 어떠한 문자열도 되돌릴 수 있게 엔코딩하여 만약 필요로 한다면 바이트 인코딩으로 돌아갈 수 있습니다.

In [0]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('인코딩된 문자열은 {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('원래 문자열: "{}"'.format(original_string))

In [0]:
assert original_string == sample_string

In [0]:
for index in encoded_string:
  print ('{} ----> {}'.format(index, encoder.decode([index])))

## 학습을 위한 데이터 준비

Next create batches of these encoded strings. Use the `padded_batch` method to zero-pad the sequences to the length of the longest string n the batch:

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [0]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

## 모델 만들기

`tf.keras.Sequential` 모델을 빌드하고 임베딩 레이어로 시작합니다. 임베딩 레이어는 단어 당 하나의 벡터를 저장합니다. 호출되면 단어 인덱스 시퀀스를 벡터 시퀀스로 변환합니다. 이 벡터는 훈련이 가능합니다. (충분한 데이터에 대해서) 훈련 후 비슷한 의미를 가진 단어는 종종 비슷한 벡터를 갖습니다.

이 인덱스-조회는 one-hot 인코딩 된 벡터를 `tf.keras.layers.Dense` 레이어를 통해 전달하는 동등한 작업보다 훨씬 효율적입니다.

순환 인공 신경망(RNN)은 요소를 반복하여 시퀀스 입력을 처리합니다. RNN은 출력을 한 단계에서 입력으로 전달한 다음에 다음 단계로 전달합니다.

`tf.keras.layers.Bidirectional` 래퍼도 RNN 레이어와 함께 사용할 수 있습니다. 이는 RNN 계층을 통해 입력을 앞뒤로 전파 한 다음 출력을 연결합니다. 이것은 RNN이 장거리 종속성을 학습하는데 도움을 줍니다.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Keras 모델을 컴파일하여 학습 과정을 구성합니다:

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## 모델 학습하기

In [0]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset)

print('테스트 손실값: {}'.format(test_loss))
print('테스트 정확도: {}'.format(test_acc))

The above model does not mask the padding applied to the sequences. This can lead to skew if trained on padded sequences and test on un-padded sequences. Ideally you would [use masking](../../guide/keras/masking_and_padding) to avoid this, but as you can see below it only have a small effect on the output.

If the prediction is >= 0.5, it is positive else it is negative.

In [0]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [0]:
def sample_predict(sentence, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [0]:
# predict on a sample text without padding.

sample_pred_text = ('영화는 좋았습니다. 애니메이션과 그래픽은 '
                    '너무나 훌륭했습니다. 나는 이 영화를 추천합니다.')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)

In [0]:
# predict on a sample text with padding

sample_pred_text = ('영화는 좋았습니다. 애니메이션과 그래픽은 '
                    '너무나 훌륭했습니다. 나는 이 영화를 추천합니다.')
predictions = sample_predict(sample_pred_text, pad=True)
print (predictions)

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')

## Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* Return either the full sequences of successive outputs for each timestep (a 3D tensor of shape `(batch_size, timesteps, output_features)`).
* Return only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)).

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [0]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset)

print('테스트 손실값: {}'.format(test_loss))
print('테스트 정확도: {}'.format(test_acc))

In [0]:
# predict on a sample text without padding.

sample_pred_text = ('영화는 좋지 않습니다. 애니메이션과 그래픽 부분이 '
                    '끔찍했습니다. 이 영화는 추천하지 않습니다.')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)

In [0]:
# predict on a sample text with padding

sample_pred_text = ('영화는 좋지 않습니다. 애니메이션과 그래픽 부분이 '
                    '끔찍했습니다. 이 영화는 추천하지 않습니다.')
predictions = sample_predict(sample_pred_text, pad=True)
print (predictions)

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')

[GRU layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)과 같은 기존의 다른 반복 계층을 확인합니다..

만약custom RNN 빌드에 관심이 있으시다면, [Keras RNN Guide](../../guide/keras/rnn.ipynb)를 보길 바랍니다..
